In [10]:
#ANALISI SERIE TEMPORALI DI DATI FINANZIARI pag 44
import yfinance as yf
import pandas as pd
import numpy as np

ticker = 'TSLA'
tkr = yf.Ticker(ticker)
df = tkr.history(period='5d')
#print(df)
#print(df['Close'])

#CALCOLARE LE VARIAZIONI PERCENTUALI
#print(pd.concat([df['Close'], df['Close'].shift(2)], axis=1, keys= ['Close', '2DaysShift']))
(df['Close'] - df['Close'].shift(2))/ df['Close'].shift(2)
df['2daysRise'] = np.log(df['Close'] / df['Close'].shift(2))
#print(df[['Close','2daysRise']])

#CALCOLI A FINESTRA MOBILE pag 46
df['2daysAvg'] = df['Close'].shift(1).rolling(2).mean()
print(df[['Close', '2daysAvg']])

#CALCOLARE LA VARIAZIONE PERCENTUALE DI UNA MEDIA MOBILE
df['2daysAvgRise'] = np.log(df['Close'] / df['2daysAvg'])
print(df[['Close','2daysRise','2daysAvgRise']])

#SERIE TEMPORALI MULTIVARIATE pag 48
stocks = pd.DataFrame() 
tickers = ['MSFT', 'TSLA', 'GM', 'AAPL', 'ORCL', 'AMZN'] 
for ticker in tickers: 
    tkr = yf.Ticker(ticker) 
    hist = tkr.history(period='5d') 
    hist = pd.DataFrame(hist[['Close']].rename(columns={'Close': ticker})) 
if stocks.empty: 
    stocks = hist 
else: 
    stocks = stocks.join(hist)

#ELABORARE SERIE TEMPORALI MULTIVARIATE pag 50
stocks_to_keep = []
for i in stocks.columns: 
    if stocks[stocks[i]/stocks[i].shift(1) < 97].empty: 
        stocks_to_keep.append(i)
print(stocks_to_keep)

#QUI STAMPIAMO il DataFrame stocks in modo che includa solo le colonne della lista stocks_to_keep
print(stocks[stocks_to_keep])

                                Close    2daysAvg
Date                                             
2025-06-06 00:00:00-04:00  295.140015         NaN
2025-06-09 00:00:00-04:00  308.579987         NaN
2025-06-10 00:00:00-04:00  326.089996  301.860001
2025-06-11 00:00:00-04:00  326.429993  317.334991
2025-06-12 00:00:00-04:00  324.445007  326.259995
                                Close  2daysRise  2daysAvgRise
Date                                                          
2025-06-06 00:00:00-04:00  295.140015        NaN           NaN
2025-06-09 00:00:00-04:00  308.579987        NaN           NaN
2025-06-10 00:00:00-04:00  326.089996   0.099724      0.077210
2025-06-11 00:00:00-04:00  326.429993   0.056234      0.028258
2025-06-12 00:00:00-04:00  324.445007  -0.005057     -0.005579
[]
Empty DataFrame
Columns: []
Index: [2025-06-06 00:00:00-04:00, 2025-06-09 00:00:00-04:00, 2025-06-10 00:00:00-04:00, 2025-06-11 00:00:00-04:00, 2025-06-12 00:00:00-04:00]


In [ ]:
#ANALIZZARE LE DIPENDENZE TRA VARIABILI pag 51
import yfinance as yf
import pandas as pd

ticker = 'TSLA'
tkr = yf.Ticker(ticker)
df = tkr.history(period='1mo')

#si riduce il DataFrame di conseguenza a 2 colonne e si cambia il nome della colonna Close in Price
df = df[['Close','Volume']].rename(columns={'Close': 'Price'})

#Qui si calcola la variazione percentuale giornaliera nella colonna Price
df['priceRise'] = np.log(df['Price'] / df['Price'].shift(1))
#crea una colonna volumeRise che mostra la variazione percentuale del volume rispetto al giorno precedente
df['volumeRise'] = np.log(df['Volume'] / df['Volume'].shift(1))
#print(df)

#stampiamo solo le righe in cui la variazione percentuale del prezzo è maggiore di 0.05 (5%)
print(df[abs(df['priceRise']) > .05])

#si calcola la variazione media del volume sull’intera serie
print(df['volumeRise'].mean().round(4))

'''si calcola la variazione media del volume solo per le righe con variazioni di prezzo
superiori alla media. Se il risultato è superiore alla variazione media del volume nell’intera
serie, si saprà che esiste una connessione tra l’aumento della volatilità e l’aumento del volume'''
print(df[abs(df['priceRise']) > .05]['volumeRise'].mean().round(4))

#aggiunta colonna volumSum per calcolo a finestra mobile
df['volumeSum'] = df['Volume'].shift(1).rolling(2).sum().fillna(0).astype(int)
print(df[abs(df['priceRise']) > .05].replace(0, np.nan).dropna())

df['nextVolume'] = df['Volume'].shift(-1).fillna(0).astype(int)
print(df[abs(df['priceRise']) > .05].replace(0, np.nan).dropna())

                                Price     Volume  priceRise  volumeRise
Date                                                                   
2025-05-27 00:00:00-04:00  362.890015  120146400   0.067097    0.350129
2025-06-05 00:00:00-04:00  284.700012  287499800  -0.153849    1.066991
2025-06-10 00:00:00-04:00  326.089996  151256500   0.055192    0.070863
-0.0558
0.496
